### Set Up

In [78]:
#Import packages
import pandas as pd
from pandas.io.json import json_normalize
import requests
import json
import time

In [116]:
url = "https://lz4.overpass-api.de/api/interpreter"
directory = 'INSERT HERE'

### Extract Data

In [70]:
#Breaking up the area I'm interested in - probably could be done more efficiently
#OSM API bbox defined by bottom left and top right corner, latitude longitude
bbox1 = "(-33.695, 150.214, -32.948, 150.754)"
bbox2 = "(-33.695, 150.754, -32.948, 151.128)"
bbox3 = "(-34.432, 149.952, -33.695, 150.653)"
bbox4 = "(-34.432, 150.653, -33.695, 151.004)"
bbox_list = [bbox1, bbox2, bbox3, bbox4]

In [71]:
results_dic = {}

In [72]:
i = 1
for bbox in bbox_list:
    
    #Query in weird query language
    query = f"""
    [out:json];
    node[amenity]{bbox};
    out;
    node[office]{bbox};
    out;
    node[building]{bbox};
    out;
    node[shop]{bbox};
    out;
    """
    
    r = requests.get(url, params={'data': query})
    data = r.json()['elements']  # read response as JSON and get the data
    
    results_dic["bbox" + str(i)] = data
    
    file_name = "bbox" + str(i) + ".json"
    
    with codecs.open( directory + file_name, 'w', 'utf-8') as f:
        f.write(r.text)
        f.close()
    
    time.sleep(10) 
    print("Step " + str(i) + " complete")
    
    i = i + 1

Step 1 complete
Step 2 complete
Step 3 complete
Step 4 complete


In [90]:
complete_table = pd.DataFrame()

In [94]:
for key in results_dic.keys():
    df = json_normalize(results_dic[key])
    frames =  [complete_table, df]
    complete_table = pd.concat(frames, sort = False)

In [96]:
complete_table.to_csv(directory + 'osm_data.csv')

### Filter the data to relevant entries
Relevant categories determined by manual search of results

In [97]:
amenity_filter = ["cafe", "fast_food", "convention_centre", "food_court", "food_hall"] 
shop_filter = ["supermarket", "greengrocer", "convenience", "butcher", "bakery",
                "deli", "general_store", "confectionery", "health_food", "chocolate",
               "wine", "seafood", "food", "gift;bakery", "pastry"]
building_filter = ["farm", "farm_auxiliary"]
filters = {"tags.amenity":amenity_filter, "tags.shop":shop_filter, "tags.building":building_filter}

In [105]:
filter_table = pd.DataFrame()
for key in filters.keys():
    temp = complete_table[complete_table[key].isin(filters[key])]
    frames=[filter_table, temp]
    filter_table = pd.concat(frames, sort = False)
    print(key)

tags.amenity
tags.shop
tags.building


In [117]:
filter_table.to_csv(directory + 'filtered_osm_data.csv')

In [114]:
#Proportion with names
no_named = len(filter_table["tags.name"][pd.notnull(filter_table["tags.name"])])
no_total = len(filter_table["tags.name"])
print(str(no_named) + '/' + str(no_total))

1794/2060
